In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import json
import time
import sqlite3
import base64
from datetime import datetime

In [2]:
## Pull Ticketmaster dataset from SQL db

# Establish SQL connection
conn = sqlite3.connect('Ticket Information.db')
curs = conn.cursor()

# Pull relevant table into DF
tm_events = pd.read_sql_query('''SELECT
                                  eventId,
                                  eventName,
                                  attractionName,
                                  venueName,
                                  venueZipCode,
                                  eventStartDateTime
                                FROM
                                  ticketmaster_events
                              ''',
                              conn
                             )

# Close connection
curs.close()
conn.close()

tm_events.shape
tm_events.head()

,eventId,eventName,attractionName,venueName,venueZipCode,eventStartDateTime
0,G5viZ469tfQS0,LSD Tour: Lucinda Williams/Steve Earle/Dwight ...,Lucinda Williams,Ascend Amphitheater part of the Journeys Conce...,37203,2018-08-12T23:30:00Z
1,1AvfZ4aGkmEPKsw,Strangelove - The Depeche Mode Experience,Strangelove,The Fillmore Silver Spring presented by Cricke...,20910,2018-08-05T00:00:00Z
2,1AyZAC7Gkd9QyRb,Legends In Concert (Las Vegas),Legends In Concert (Las Vegas),Donny & Marie Showroom at Flamingo Las Vegas,89109,2018-09-28T04:30:00Z
3,1AyZAC7GkdKbzEd,Legends In Concert (Las Vegas),Legends In Concert (Las Vegas),Donny & Marie Showroom at Flamingo Las Vegas,89109,2018-07-26T23:00:00Z
4,G5viZ4A2p8MNN,Metallica - WorldWired Tour,Metallica,Verizon Arena,72114,2019-01-21T01:30:00Z


In [3]:
## Call API to retrieve access token 

# Concatenate API Key & API Secret and encode
key_secret = b'LbrjYwRQtqMjHx0dtsf2aPJXRAYa:GOrvYTqM_ITZ6dwOzdCWWjLOJ4Ia'
authorization_token = base64.b64encode(key_secret)
headers = {'Authorization':'Basic {}'.format(authorization_token),
           'Content-Type':'application/x-www-form-urlencoded'
          }

# Set Params
params = {'grant_type':'password',
          'username':'albert.troszczynski@gmail.com',
          'password':'p@8QZHD576!z',
          'scope':'PRODUCTION',
          'client_id':'LbrjYwRQtqMjHx0dtsf2aPJXRAYa',
          'client_secret':'GOrvYTqM_ITZ6dwOzdCWWjLOJ4Ia'
         }

# Call API
response = requests.post('https://api.stubhub.com/login', headers=headers, params=params)
response.text

'{"access_token":"d3098739-24b4-3fa9-8f98-d09cf37b1ec8","refresh_token":"cefaa2cc-8832-3023-8b7b-ac6e0abcb9f5","scope":"default","token_type":"Bearer","expires_in":14844067}'

In [4]:
## Create static API Token variable

# Set access token
access_token = 'd3098739-24b4-3fa9-8f98-d09cf37b1ec8'

# Set Headers
headers = {'Authorization':'Bearer {}'.format(access_token)}

In [5]:
## Nested for loops to extract all event data

# Create empty DF
sh_events = pd.DataFrame()

# Create list of all zip codes
zip_codes = list(tm_events['venueZipCode'].unique())
zip_codes.remove('89109')  # Remove this Vegas zip code that is problematic for joins
idx_multiplier = len(zip_codes)//10

# For loop to break up zip code API calls based on request limitations
for i in range(10):
    if i != 9:
        zip_pull_request = ' |'.join(zip_codes[i*idx_multiplier:(i+1)*idx_multiplier])
    else:
        zip_pull_request = ' |'.join(zip_codes[i*idx_multiplier:])
    # Set params
    params = {'postalCode':zip_pull_request,
              'rows':'500',                   # Maximum rows per pull
              'start':'0',                    # Starting at row 0
              'parking':False                 # Removing parking 'events'
             }
    # Call API
    response = requests.get('https://api.stubhub.com/search/catalog/events/v3', headers=headers, params=params)
    # Extract JSON to DF
    response_json = json_normalize(response.json())
    events = json_normalize(response_json['events'][0])[['eventDateUTC','id','venue.postalCode','status']]
    sh_events = sh_events.append(events)
    # Print loop count for tracking
    print('Zip loop {} of 10'.format(i+1))
    # API allows 10 calls per minute - sleep to prevent API rejection
    time.sleep(6)
    # Set total event count
    row_count = response_json['numFound'][0]
    # For loop to extract the rest of the rows from API call
    for row_start in range(500, row_count, 500):
        # Set params
        params = {'postalCode':zip_pull_request,
                  'rows':'500',                   # Maximum rows per pull
                  'start':str(row_start),         # Starting at row 0
                  'parking':False                 # Removing parking 'events'
                 }
        # Call API
        response = requests.get('https://api.stubhub.com/search/catalog/events/v3', headers=headers, params=params)
        # Extract JSON to DF
        response_json = json_normalize(response.json())
        events = json_normalize(response_json['events'][0])[['eventDateUTC','id','venue.postalCode','status']]
        sh_events = sh_events.append(events)
        # Print row count for tracking
        print ('Row count {} of {}'.format(row_start, row_count))
        # API allows 10 calls per minute - sleep to prevent API rejection
        time.sleep(6)
        
print(sh_events.shape)
sh_events.head()

Zip loop 1 of 10
Row count 500 of 12591
Row count 1000 of 12591
Row count 1500 of 12591
Row count 2000 of 12591
Row count 2500 of 12591
Row count 3000 of 12591
Row count 3500 of 12591
Row count 4000 of 12591
Row count 4500 of 12591
Row count 5000 of 12591
Row count 5500 of 12591
Row count 6000 of 12591
Row count 6500 of 12591
Row count 7000 of 12591
Row count 7500 of 12591
Row count 8000 of 12591
Row count 8500 of 12591
Row count 9000 of 12591
Row count 9500 of 12591
Row count 10000 of 12591
Row count 10500 of 12591
Row count 11000 of 12591
Row count 11500 of 12591
Row count 12000 of 12591
Row count 12500 of 12591
Zip loop 2 of 10
Row count 500 of 9056
Row count 1000 of 9056
Row count 1500 of 9056
Row count 2000 of 9056
Row count 2500 of 9056
Row count 3000 of 9056
Row count 3500 of 9056
Row count 4000 of 9056
Row count 4500 of 9056
Row count 5000 of 9056
Row count 5500 of 9056
Row count 6000 of 9056
Row count 6500 of 9056
Row count 7000 of 9056
Row count 7500 of 9056
Row count 8000 of

,eventDateUTC,id,venue.postalCode,status
0,2018-08-13T23:00:00+0000,103714850,10036,Active
1,2018-08-14T01:30:00+0000,103714851,10036,Active
2,2018-08-28T01:30:00+0000,103713659,10036,Active
3,2018-08-27T01:30:00+0000,103713657,10036,Active
4,2018-08-27T23:00:00+0000,103713658,10036,Active


In [6]:
## Convert datetime strings for joins

def sh_str_to_date (x):
    x = x[:-5]
    return datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')

def tm_str_to_date (x):
    return datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')

sh_events['converted_startDate'] = sh_events.eventDateUTC.apply(sh_str_to_date)
tm_events = tm_events[tm_events['eventStartDateTime'].astype(bool)]
tm_events['converted_startDate'] = tm_events.eventStartDateTime.apply(tm_str_to_date)

In [7]:
## Merge tables

# Rename columns for merge
sh_events.rename(columns={'venue.postalCode':'venueZipCode', 'id':'sh_Id', 
                          'name':'sh_name', 'venue.name':'sh_venue'},
                 inplace=True
                )

# Merge on start date and zip code
sh_events = tm_events.merge(sh_events, how='left', on=['venueZipCode','converted_startDate'])

# Drop duplicates and missing data 
sh_events.dropna(subset=['sh_id'], inplace=True)
sh_events.drop_duplicates(subset=['sh_id','eventId'], inplace=True)
sh_events.shape

(6256, 10)

In [18]:
sh_events

,eventId,eventName,attractionName,venueName,venueZipCode,eventStartDateTime,converted_startDate,eventDateUTC,sh_Id,status
0,G5viZ469tfQS0,LSD Tour: Lucinda Williams/Steve Earle/Dwight ...,Lucinda Williams,Ascend Amphitheater part of the Journeys Conce...,37203,2018-08-12T23:30:00Z,2018-08-12 23:30:00,2018-08-12T23:30:00+0000,103536569.0,Active
3,1AvfZ4aGkmEPKsw,Strangelove - The Depeche Mode Experience,Strangelove,The Fillmore Silver Spring presented by Cricke...,20910,2018-08-05T00:00:00Z,2018-08-05 00:00:00,2018-08-05T00:00:00+0000,103669015.0,Active
6,G5viZ4A2p8MNN,Metallica - WorldWired Tour,Metallica,Verizon Arena,72114,2019-01-21T01:30:00Z,2019-01-21 01:30:00,2019-01-21T01:30:00+0000,103491774.0,Active
11,vv1AaZA8MGkdobIGv,Gino Vannelli,Gino Vannelli,The Libbey Bowl,93023,2018-08-05T02:00:00Z,2018-08-05 02:00:00,2018-08-05T02:00:00+0000,103678874.0,Active
15,vvG1FZ4u5v6MNa,SiriusXM Outlaw Country Presents: Turnpike Tro...,Turnpike Troubadours,Theatre of Living Arts,19147,2018-10-20T01:00:00Z,2018-10-20 01:00:00,2018-10-20T01:00:00+0000,103703415.0,Active
16,vvG1YZ4FN4RVj_,SiriusXM Pres. Goo Goo Dolls - Dizzy Up The Gi...,Goo Goo Dolls,House of Blues Dallas presented by Cricket Wir...,75202,2018-10-04T00:00:00Z,2018-10-04 00:00:00,2018-10-04T00:00:00+0000,103567248.0,Active
20,vvG1bZ4Gh5VfTj,Beck,Beck,Armory,55415,2018-09-14T01:00:00Z,2018-09-14 01:00:00,2018-09-14T01:00:00+0000,103670475.0,Active
22,G5viZ41riGAZy,Nine Inch Nails: Cold and Black and Infinite N...,nine inch nails,The Orpheum Theatre Memphis,38103,2018-09-25T00:00:00Z,2018-09-25 00:00:00,2018-09-25T00:00:00+0000,103606625.0,Active
24,vvG10Z4uBJBeLJ,The Expendables,The Expendables,House of Blues Anaheim presented by Cricket Wi...,92802,2018-09-01T02:00:00Z,2018-09-01 02:00:00,2018-09-01T02:00:00+0000,103733512.0,Active
28,G5dIZ4GV4up99,St. Paul and the Broken Bones,St. Paul and the Broken Bones,House of Blues Houston presented by Cricket Wi...,77002,2018-09-16T00:00:00Z,2018-09-16 00:00:00,2018-09-16T00:00:00+0000,103704747.0,Active


In [17]:
## Pull ticket listings info for each event

# Set empty DF
listings_pull = pd.DataFrame()

for i, (sh_id, tm_id) in enumerate(zip(sh_events.sh_Id, sh_events.eventId)):
    params = {'eventId':str(int(sh_id)),
              'pricingsummary':'true'
             }
    response = requests.get('https://api.stubhub.com/search/inventory/v2', headers=headers, params=params)
    response_json = json_normalize(response.json())
    # Pulling information of event is available
    if 'listing' in response_json.columns:
        # Organizing relevant information
        listings = json_normalize(response_json['listing'][0])
        listings['sh_Id'] = response_json['eventId'][0]
        listings['maxQuantity'] = response_json['maxQuantity'][0]
        listings['minQuantity'] = response_json['minQuantity'][0]
        listings['totalListings'] = response_json['totalListings'][0]
        listings['totalTickets'] = response_json['totalTickets'][0]
        listings['datetime'] = datetime.utcnow()
        listings['tm_Id'] = tm_id
        # Renaming columns for SQL dump
        listings.rename(columns={'currentPrice.amount':'currentPrice','currentPrice.currency':'currency'},
                        inplace=True)
        listings_pull = listings_pull.append(listings)
        print(i, sh_id)
        
 # Adjusting lists to strings so the can be properly added to SQL DB
listings_pull['deliveryTypeList'] = listings_pull.deliveryTypeList.astype(str)
listings_pull['deliveryMethodList'] = listings.deliveryMethodList.astype(str)
listings_pull['splitVector'] = listings_pull.splitVector.astype(str)
listings_pull['listingAttributeList'] = listings_pull.listingAttributeList.astype(str)
listings_pull['listingAttributeCategoryList'] = listings_pull.listingAttributeCategoryList.astype(str)

# Append listings info to SQL DB
conn = sqlite3.connect('Ticket Information.db')
listings_pull.to_sql('stubhub_listings', conn, if_exists='append')
conn.close()

0 103536569.0
2 103491774.0
3 103678874.0
4 103703415.0
5 103567248.0
6 103670475.0
7 103606625.0
8 103733512.0
9 103704747.0
10 103657774.0
11 103718130.0
12 103653872.0
13 103677852.0
14 103487492.0
15 103629843.0
16 103721940.0
19 103585957.0
20 103665119.0
21 103642544.0
22 103529066.0
23 103668193.0
24 103590897.0
25 103433169.0
26 103270902.0
27 103447276.0
28 103595134.0
29 103659319.0
30 103556571.0
31 103580606.0
32 103607673.0
33 103740249.0
34 103736723.0
35 103585987.0
36 103684225.0
37 103502419.0
38 103535122.0
39 103504431.0
41 103654061.0
42 103489370.0
43 103614308.0
44 103579308.0
45 103662902.0
46 103727631.0
47 103436979.0
48 103655188.0
49 103699718.0
50 103728422.0
51 103725230.0
53 103718130.0
54 103653891.0
55 103556389.0
56 103602435.0
57 103583695.0
58 103618719.0
59 103529248.0
60 103561932.0
62 103652308.0
63 103726168.0
64 103640251.0
65 103668885.0
66 103655411.0
67 103566157.0
68 103655992.0
69 103726961.0
70 103299224.0
71 103562187.0
72 103644793.0
73 1

561 103638214.0
562 103659019.0
563 103626348.0
564 103454492.0
565 103296967.0
566 103512253.0
567 103536138.0
568 103478854.0
569 103413789.0
570 103469065.0
571 103688541.0
572 103676784.0
573 103582866.0
574 103503890.0
575 103658088.0
576 103745208.0
577 103722689.0
578 103686159.0
579 103679593.0
580 103716051.0
581 103715999.0
582 103665282.0
583 103677360.0
584 103658789.0
585 103669535.0
586 103668444.0
587 103632476.0
588 103556541.0
589 103270922.0
590 103607552.0
591 103670636.0
592 103602487.0
593 103608730.0
594 103428114.0
595 103591583.0
596 103599987.0
597 103616385.0
598 103411717.0
599 103711673.0
600 103431483.0
601 103502619.0
602 103685774.0
603 103753723.0
604 103727925.0
605 103501090.0
606 103646061.0
607 103668820.0
608 103297704.0
609 103631349.0
610 103565706.0
612 103607237.0
613 103668434.0
614 103468485.0
615 103586036.0
616 103586342.0
617 103602105.0
618 103532812.0
619 103502579.0
620 103595426.0
621 103604063.0
622 103703092.0
624 103726252.0
625 1035

1103 103413825.0
1104 103616284.0
1105 103530777.0
1106 103583792.0
1108 103426651.0
1109 103725059.0
1110 103653842.0
1111 103607652.0
1112 103668814.0
1113 103726934.0
1114 103655944.0
1115 103594068.0
1116 103495052.0
1117 103299148.0
1118 103527202.0
1119 103158885.0
1120 103566105.0
1121 103024543.0
1122 103647391.0
1123 103379625.0
1124 103645672.0
1125 103678369.0
1126 103499601.0
1127 103457140.0
1128 103436950.0
1129 103493909.0
1130 103679729.0
1131 103258394.0
1132 103602749.0
1133 103663024.0
1134 103586382.0
1135 103487571.0
1136 103602113.0
1137 103590364.0
1138 103532819.0
1139 103726949.0
1140 103736817.0
1141 103723981.0
1142 103493720.0
1143 103561365.0
1144 103595681.0
1145 103659042.0
1146 103571555.0
1147 103726152.0
1148 103489468.0
1149 103195616.0
1150 103371884.0
1151 103132902.0
1152 103594957.0
1153 103512388.0
1154 103512140.0
1155 103592515.0
1156 103701965.0
1157 103580235.0
1158 103582866.0
1159 103592526.0
1160 103596574.0
1161 103752312.0
1162 103735500

1620 103665346.0
1621 103672533.0
1622 103700066.0
1623 103602131.0
1624 103632458.0
1625 103561396.0
1626 103591928.0
1627 103655119.0
1628 103602379.0
1629 103428061.0
1630 103703439.0
1631 103527152.0
1632 103604848.0
1633 103636594.0
1634 103722081.0
1635 103648406.0
1636 103608584.0
1637 103480426.0
1638 103686281.0
1640 103493742.0
1641 103480706.0
1642 103640240.0
1643 103580027.0
1644 103416605.0
1645 103669454.0
1646 103027348.0
1647 103450930.0
1648 103503886.0
1649 103680148.0
1650 103724604.0
1651 103626053.0
1653 103595484.0
1654 103518308.0
1655 103668319.0
1656 103601176.0
1657 103669434.0
1658 103493907.0
1659 103502003.0
1660 103586061.0
1661 103502410.0
1662 103634918.0
1663 103734740.0
1664 103560668.0
1665 103728425.0
1666 103661552.0
1667 103670863.0
1668 103636140.0
1669 103723352.0
1670 103565671.0
1671 103624716.0
1672 103613545.0
1673 103672584.0
1674 103585991.0
1675 9861848.0
1676 103705978.0
1677 103383455.0
1678 103176723.0
1679 103659011.0
1680 103417094.0

2135 103529017.0
2136 103289963.0
2137 103714198.0
2138 103457180.0
2139 103736579.0
2140 103532826.0
2141 103259184.0
2143 103614324.0
2144 103663791.0
2145 103234102.0
2146 103495000.0
2147 103609843.0
2148 103677842.0
2149 103501974.0
2150 103602093.0
2151 103672931.0
2152 103700096.0
2154 103592459.0
2155 103684665.0
2156 103593661.0
2157 103729553.0
2158 103639979.0
2159 103510424.0
2160 103736347.0
2161 103626056.0
2162 103722703.0
2164 103593629.0
2165 103596552.0
2166 103728454.0
2167 103397203.0
2168 103635965.0
2169 103615498.0
2170 103614116.0
2171 103426578.0
2172 103660392.0
2173 103527491.0
2174 103468423.0
2175 103559620.0
2176 103720809.0
2177 103602358.0
2178 103663017.0
2179 103661750.0
2180 103738063.0
2181 103735407.0
2182 103738892.0
2183 103721475.0
2184 103665709.0
2185 103702270.0
2186 103572914.0
2187 103737194.0
2188 103614741.0
2189 103654003.0
2190 103639987.0
2191 103618583.0
2192 103560653.0
2193 103527143.0
2194 103723912.0
2195 103585035.0
2196 103556596

...

4170 103592698.0
4171 103669458.0
4172 103642538.0
4173 103706004.0
4174 103383492.0
4175 103571487.0
4176 103454520.0
4177 103669818.0
4178 103643947.0
4179 103331523.0
4180 103526997.0
4181 103413817.0
4182 103512282.0
4183 103728387.0
4184 103319260.0
4185 103653847.0
4186 103592014.0
4187 103653761.0
4188 103027355.0
4189 103575415.0
4190 103450908.0
4191 103302566.0
4192 103640115.0
4193 103468492.0
4194 103198788.0
4195 103491795.0
4196 103663008.0
4197 103665462.0
4198 103671004.0
4199 103580793.0
4200 103612795.0
4201 103679487.0
4202 103474755.0
4203 103423561.0
4204 103723546.0
4205 103580606.0
4206 103607673.0
4207 103709768.0
4208 103716057.0
4209 103721235.0
4210 103659911.0
4211 103590412.0
4212 103665357.0
4213 103592523.0
4214 103632574.0
4215 103722073.0
4216 103613538.0
4217 103560399.0
4218 103412408.0
4219 103457200.0
4220 103731055.0
4221 103579242.0
4222 103619922.0
4223 103671044.0
4224 103318746.0
4225 103468523.0
4226 103703913.0
4227 103733252.0
4228 103636775

4680 103560070.0
4681 103673321.0
4682 103612397.0
4684 103680158.0
4685 103681972.0
4686 103560669.0
4687 103660277.0
4688 103432973.0
4689 103521257.0
4692 103530554.0
4693 103663589.0
4695 103724427.0
4696 103468436.0
4697 103736471.0
4698 103671066.0
4699 103509664.0
4701 103560943.0
4702 103594067.0
4703 103563641.0
4704 103591643.0
4706 103682887.0
4707 103680133.0
4708 103668487.0
4709 103668442.0
4711 103705142.0
4712 103654269.0
4713 103627235.0
4714 103749331.0
4716 103680160.0
4717 103685198.0
4718 103397185.0
4719 103653848.0
4720 103612156.0
4721 103640518.0
4722 103600144.0
4723 103570993.0
4724 103718015.0
4725 103482565.0
4726 103677695.0
4727 103436975.0
4728 103705382.0
4729 103655420.0
4730 103668891.0
4731 103660299.0
4732 103299232.0
4733 103647747.0
4734 103672682.0
4735 103727897.0
4737 103578142.0
4738 103596415.0
4739 103668852.0
4740 103666356.0
4741 103648482.0
4742 103655968.0
4743 103496042.0
4745 103566129.0
4746 103566646.0
4747 103647641.0
4748 103727881

5207 103499892.0
5208 103628343.0
5209 103707734.0
5210 103605728.0
5211 103630247.0
5212 103665338.0
5213 103604998.0
5214 103436888.0
5215 103646038.0
5216 103516245.0
5217 103563626.0
5218 103668955.0
5219 103053082.0
5220 103558342.0
5221 103679412.0
5222 103580920.0
5223 103658751.0
5224 103635636.0
5225 103645674.0
5226 103605660.0
5227 103729597.0
5228 103584021.0
5229 103526861.0
5230 103349807.0
5231 103751372.0
5232 103614778.0
5233 103670744.0
5234 103729300.0
5235 103659600.0
5236 103660396.0
5237 103556511.0
5240 103591529.0
5241 103489013.0
5242 103371876.0
5243 103132894.0
5244 103024454.0
5245 103498450.0
5246 103594949.0
5247 103362894.0
5248 103512132.0
5249 103583886.0
5250 103723452.0
5251 103587454.0
5252 103642757.0
5253 103685784.0
5254 103602377.0
5255 103495535.0
5256 103659586.0
5257 103532752.0
5258 103635634.0
5259 103706148.0
5260 103652813.0
5261 103583644.0
5262 103592466.0
5263 103570943.0
5264 103732425.0
5265 103669960.0
5266 103427329.0
5267 103592661

5726 103571055.0
5727 103702061.0
5730 103729516.0
5732 103607604.0
5733 103665165.0
5735 103498211.0
5736 103709496.0
5737 103728386.0
5738 103130756.0
5739 103668822.0
5740 103726916.0
5741 103669662.0
5742 103673428.0
5743 103566086.0
5744 103297705.0
5745 103647418.0
5746 103158866.0
5747 103495038.0
5748 103498554.0
5749 103655926.0
5750 103612018.0
5751 103598578.0
5753 103572625.0
5754 103612488.0
5755 103553292.0
5756 103603997.0
5757 103605197.0
5758 103259176.0
5759 103733621.0
5760 103672961.0
5761 103602354.0
5762 103541263.0
5764 103618636.0
5765 103687047.0
5766 103594065.0
5767 103648561.0
5768 103436946.0
5770 103593401.0
5771 103558557.0
5772 103707913.0
5773 103676687.0
5774 103736746.0
5775 103663781.0
5776 103592693.0
5777 103536201.0
5778 103640464.0
5779 103593841.0
5780 103703926.0
5781 103721285.0
5782 103586050.0
5783 103532807.0
5784 103594848.0
5785 103530318.0
5786 103590410.0
5787 103560935.0
5788 103270859.0
5789 103468719.0
5791 103677665.0
5792 103587440

6249 103533781.0
6250 103607666.0
6252 103712335.0
6253 103399991.0
6254 103671920.0
6255 103498223.0
